In [34]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
import scipy
from scipy.special import expit
from generate_data import get_sampler, get_missing_data_sample
from missing_data import add_cumulative_estimators, add_cumulative_estimators_pred, mean_estimator_plotter, get_mean_estimators, get_mean_estimators_pred, eval_estimators
from numpy.random import default_rng
import seaborn as sns
#sns.set_theme()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [36]:
# Distribution parameters
sigma = 1
sample_size = 1000

In [37]:
random_seed = 1000
r = default_rng(random_seed)
sampler = get_sampler(dist="SeaVan1", sigma=sigma)
d = sampler(sample_size)
sns.jointplot(data=d, x="x", y="y", kind="reg",x_jitter=.1, scatter_kws={"s": 1})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

A sample from the data generating distribution is plotted above. (The x values in the scatter plot are jittered so it's easier to see the distribution.) Our objective is to estimate the average value of y for this dataset. Unfortunately, we only get a biased sample. When obs is true, we observe x and y, but when obs is false, we only observe x. Below is a plot of the observed data:

In [38]:
d

,x,obs_prob,obs,y,obs_prob_pred
0,1,0.500000,True,0.560020,0.482226
1,1,0.500000,False,1.314556,0.482226
2,1,0.500000,True,-0.239596,0.482226
3,2,0.017986,False,3.012469,0.023310
4,0,0.982014,True,-0.376363,0.973222
...,...,...,...,...,...
995,2,0.017986,False,2.259842,0.023310
996,0,0.982014,True,-1.310659,0.973222
997,0,0.982014,True,0.426883,0.973222
998,2,0.017986,False,1.848257,0.023310


obs_prob is the propensity generated by using the expit function and obs_prob_pred is the one learned using Logistic Regression.

In [39]:
sns.jointplot(data=d[d["obs"]], x="x", y="y", kind="reg",x_jitter=.1, scatter_kws={"s": 1})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Below we take a large sample to get a good estimate of the true population mean:

In [56]:
d_big = sampler(n=10000000)
seavan1_full_data_mean = np.mean(d_big['y'])
print(f"Mean of all y's from sample of size {d_big.shape[0]}: {seavan1_full_data_mean} ")

Mean of all y's from sample of size 10000000: 1.000250092421415 


If we naively take the mean of the observed y's, we get a heavily biased estimate:

In [57]:
obs_mean = np.mean(d[d["obs"]]['y'])
print(f"Mean of all _observed_ y's from sample of size {d.shape[0]}: {obs_mean} ")

Mean of all _observed_ y's from sample of size 1000: 0.3082255149167164 


Now we'll generate some other estimators and see how they do.

In [49]:
#expit propensity
sampler = get_sampler(dist="SeaVan1", sigma=sigma, rng = default_rng(1000))
d = sampler(sample_size)
d = add_cumulative_estimators(d)
fig, axs = plt.subplots(2,1, sharex='col')
mean_estimator_plotter(d=d, id_col="n", value_cols=["ipw_est"], 
                       true_mean=seavan1_full_data_mean, include_scatter=False, 
                       param_dict={"markers":False}, ax=axs[0])

sns.scatterplot(data=d[d["obs"]], x="n", y="y", hue = "obs_prob", size="weight", ax=axs[1], sizes=(3, 100), legend=False) #

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
d

,x,obs_prob,obs,y,obs_prob_pred,n,cum_obs_cnt,cum_obs_val,obs_mean,weight,cum_obs_weight,cum_obs_weighted_y,ipw_est,ipw_est_b,ipw_est_b_b,ipw_est_b3
0,0,0.982014,True,3.243483,0.978823,1,1,3.243483,3.243483,1.018316,1.018316,3.302889,3.302889,3.243483,3.185145,3.127856
1,1,0.500000,False,0.167638,0.514341,2,1,3.243483,3.243483,2.000000,1.018316,3.302889,1.651445,3.243483,6.370289,12.511424
2,2,0.017986,False,1.911987,0.023691,3,1,3.243483,3.243483,55.598150,1.018316,3.302889,1.100963,3.243483,9.555434,28.150704
3,1,0.500000,False,0.097804,0.514341,4,1,3.243483,3.243483,2.000000,1.018316,3.302889,0.825722,3.243483,12.740579,50.045696
4,2,0.017986,False,1.445201,0.023691,5,1,3.243483,3.243483,55.598150,1.018316,3.302889,0.660578,3.243483,15.925723,78.196400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,0.500000,True,2.186609,0.514341,996,498,177.941570,0.357312,2.000000,1059.011423,1256.216664,1.261262,1.186216,1.115636,1.049256
996,1,0.500000,False,0.475430,0.514341,997,498,177.941570,0.357312,2.000000,1059.011423,1256.216664,1.259997,1.186216,1.116756,1.051364
997,0,0.982014,True,-0.937456,0.978823,998,499,177.004115,0.354718,1.018316,1060.029739,1255.262038,1.257778,1.184176,1.114882,1.049642
998,1,0.500000,True,1.812067,0.514341,999,500,178.816182,0.357632,2.000000,1062.029739,1258.886173,1.260146,1.185359,1.115010,1.048836


# SeaVan1

In [50]:
#log_reg propensity
sampler = get_sampler(dist="SeaVan1", sigma=sigma, rng = default_rng(1000))
d1 = sampler(sample_size)
d1 = add_cumulative_estimators_pred(d1)
fig, axs = plt.subplots(2,1, sharex='col')
mean_estimator_plotter(d=d1, id_col="n", value_cols=["ipw_est_pred"], 
                       true_mean=seavan1_full_data_mean, include_scatter=False, 
                       param_dict={"markers":False}, ax=axs[0])

sns.scatterplot(data=d1[d1["obs"]], x="n", y="y", hue = "obs_prob_pred", size="weight_pred", ax=axs[1], sizes=(3, 100), legend=False) #

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
d1

,x,obs_prob,obs,y,obs_prob_pred,n,cum_obs_cnt,cum_obs_val,obs_mean,weight_pred,cum_obs_weight_pred,cum_obs_weighted_y_pred,ipw_est_pred,ipw_est_b_pred,ipw_est_b_b_pred,ipw_est_b3_pred
0,0,0.982014,True,3.243483,0.978823,1,1,3.243483,3.243483,1.021635,1.021635,3.313655,3.313655,3.243483,3.174797,3.107565
1,1,0.500000,False,0.167638,0.514341,2,1,3.243483,3.243483,1.944235,1.021635,3.313655,1.656827,3.243483,6.349593,12.430261
2,2,0.017986,False,1.911987,0.023691,3,1,3.243483,3.243483,42.210552,1.021635,3.313655,1.104552,3.243483,9.524390,27.968087
3,1,0.500000,False,0.097804,0.514341,4,1,3.243483,3.243483,1.944235,1.021635,3.313655,0.828414,3.243483,12.699187,49.721044
4,2,0.017986,False,1.445201,0.023691,5,1,3.243483,3.243483,42.210552,1.021635,3.313655,0.662731,3.243483,15.873983,77.689132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,0.500000,True,2.186609,0.514341,996,498,177.941570,0.357312,1.944235,956.706424,1023.398684,1.027509,1.069710,1.113645,1.159384
996,1,0.500000,False,0.475430,0.514341,997,498,177.941570,0.357312,1.944235,956.706424,1023.398684,1.026478,1.069710,1.114763,1.161714
997,0,0.982014,True,-0.937456,0.978823,998,499,177.004115,0.354718,1.021635,957.728059,1022.440947,1.024490,1.067569,1.112460,1.159238
998,1,0.500000,True,1.812067,0.514341,999,500,178.816182,0.357632,1.944235,959.672294,1025.964032,1.026991,1.069077,1.112889,1.158495


In [52]:
#expit propensity
fig, axs = plt.subplots(2,1, sharex='col')
mean_estimator_plotter(d=d, id_col="n", value_cols=["ipw_est", "ipw_est_b"], 
                       true_mean=seavan1_full_data_mean, include_scatter=False, 
                       param_dict={"markers":False}, ax=axs[0])

sns.scatterplot(data=d[d["obs"]], x="n", y="y", hue = "obs_prob", size="weight", 
                ax=axs[1], sizes=(3, 100), legend=False) #

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [53]:
#log_reg propensity
fig, axs = plt.subplots(2,1, sharex='col')
mean_estimator_plotter(d=d1, id_col="n", value_cols=["ipw_est_pred", "ipw_est_b_pred"], 
                       true_mean=seavan1_full_data_mean, include_scatter=False, 
                       param_dict={"markers":False}, ax=axs[0])

sns.scatterplot(data=d1[d1["obs"]], x="n", y="y", hue = "obs_prob_pred", size="weight_pred", 
                ax=axs[1], sizes=(3, 100), legend=False) #

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [85]:
#expit propensity
sampler = get_sampler(dist="SeaVan1", sigma=sigma, rng = default_rng(5))
num_repeats = 5000  #use 5000 for better SEs
data_list = []
num_obs_list = []
for i in range(num_repeats):
    ## Randomly choose observations
    d = sampler(n=sample_size)
    est = get_mean_estimators(vals=d['y'],
                              covar=d[["x"]],
                              propensities=d["obs_prob"],
                              obs=d['obs'])
    data_list.append(est)
    num_obs_list.append(d['obs'].sum())
results_seavan1 = pd.DataFrame(data_list)
#results = results_seavan1
#estimators_to_keep = ["ipw_mean", "ipw_b_mean", "ipw_b_b_mean"]
#a = results[estimators_to_keep].apply(eval_estimators, axis=0, raw=True, 
#                                      result_type="expand", true_val=seavan1_full_data_mean)
#results_eval = pd.DataFrame(a.to_dict())
#print(results_eval)

In [86]:
#log_reg propensity
sampler = get_sampler(dist="SeaVan1", sigma=sigma, rng = default_rng(5))
num_repeats = 5000  #use 5000 for better SEs
data_list = []
num_obs_list = []
for i in range(num_repeats):
    ## Randomly choose observations
    d = sampler(n=sample_size)
    est = get_mean_estimators_pred(vals=d['y'],
                              covar=d[["x"]],
                              propensities=d["obs_prob_pred"],
                              obs=d['obs'])
    data_list.append(est)
    num_obs_list.append(d['obs'].sum())
results_seavan1_pred = pd.DataFrame(data_list)
#results = results_seavan1_pred
#estimators_to_keep = ["ipw_mean_pred", "ipw_b_mean_pred", "ipw_b_b_mean_pred"]
#a = results[estimators_to_keep].apply(eval_estimators, axis=0, raw=True, 
#                                      result_type="expand", true_val=seavan1_full_data_mean)
#results_eval = pd.DataFrame(a.to_dict())
#print(results_eval)

In [87]:
results = pd.concat([results_seavan1, results_seavan1_pred], axis=1)

estimators_to_keep = ["ipw_mean", "ipw_mean_pred", "ipw_b_mean", "ipw_b_mean_pred", "ipw_b_b_mean", "ipw_b_b_mean_pred"]
a = results[estimators_to_keep].apply(eval_estimators, axis=0, raw=True, 
                                      result_type="expand", true_val=seavan1_full_data_mean)
results_eval = pd.DataFrame(a.to_dict())
print(results_eval)

              ipw_mean  ipw_mean_pred  ipw_b_mean  ipw_b_mean_pred  \
mean          0.995142       0.828959    0.978119         0.897344   
SD            0.308634       0.179947    0.197319         0.151429   
SE            0.004365       0.002545    0.002791         0.002142   
bias         -0.005108      -0.171291   -0.022131        -0.102906   
mean_abs_err  0.244294       0.205381    0.158224         0.146352   
RMS error     0.308677       0.248438    0.198556         0.183086   

              ipw_b_b_mean  ipw_b_b_mean_pred  
mean              0.978854           0.976601  
SD                0.145780           0.137767  
SE                0.002062           0.001948  
bias             -0.021396          -0.023650  
mean_abs_err      0.116468           0.109722  
RMS error         0.147342           0.139782  


Comparison

The bias is relatively small for the trained estimators. The respective learned estimators have lower SE compared to their trained counterparts and its lower for the self-normalised and even lower for the twice self-normalised estimators. 

The twice self-normalized estimator has improved performance, presumably by removing the rest of the variation caused by correlation between the total weight and the estimate. More precisely, conditional on the total weight, the ipw and even the ipw_b estimators have a significant bias that is positively correlated with the weight. When we combine all these estimates together, the positive and negative biases inflate the overall variance of the estimator. I think the takeaway here is that there is no estimator that is "best" for all distributions. The ipw_b_b estimator happens to work quite well for this distribution, because even self-normalizing didn't review all of the correlation with the weight. But this isn't the case for other distributions.

We now consider a different kind of estimator: imputation estimators. The approach we consider here is to build a model for the expectation of Y given X, using the complete cases. Then impute the predictions for the incomplete cases. We then take the average of the Y's and imputed Y's, as though it were the full data set. For illustration, below we consider a linear fit of the data.

In [88]:
estimators_to_keep = ["ipw_mean", "ipw_mean_pred", "ipw_b_mean", "ipw_b_mean_pred", "impute_missing", "impute_missing_pred", 
                      "impute_missing_w", "impute_missing_w_pred"]
a = results[estimators_to_keep].apply(eval_estimators, axis=0, raw=True, 
                                      result_type="expand", true_val=seavan1_full_data_mean)
results_eval = pd.DataFrame(a.to_dict())
print(results_eval)


              ipw_mean  ipw_mean_pred  ipw_b_mean  ipw_b_mean_pred  \
mean          0.995142       0.828959    0.978119         0.897344   
SD            0.308634       0.179947    0.197319         0.151429   
SE            0.004365       0.002545    0.002791         0.002142   
bias         -0.005108      -0.171291   -0.022131        -0.102906   
mean_abs_err  0.244294       0.205381    0.158224         0.146352   
RMS error     0.308677       0.248438    0.198556         0.183086   

              impute_missing  impute_missing_pred  impute_missing_w  \
mean                0.998938             0.998938          0.997538   
SD                  0.077668             0.077668          0.148349   
SE                  0.001098             0.001098          0.002098   
bias               -0.001312            -0.001312         -0.002712   
mean_abs_err        0.061657             0.061657          0.116538   
RMS error           0.077679             0.077679          0.148374   

           

Comparison 

impute_missing and impute_missing_pred have the same performance since both use the same set of variables. However, impute_missing_w_pred is performing slightly better than impute_missing_w (lower SE and errors)


With impute_missing, we build a regression estimator for the missing values. The regression estimator has only 2 degrees of freedom and roughly sample_size/2 observations. This will lead to very low variance estimate of the regression model. However, more is reflected in SD than just that variance. In general, even with infinite data, the regression prediction may be wrong for certain x's because of model bias -- i.e. the true conditional distribution is not represented well in our class of models. These errors, while a consistent "bias" for individual x's, just look like noise when averaged across the x's. Thus this kind of error may also show up in the SD. If the errors for each x do not cancel out, on average, then we will end up with a bias. We will see this happening when we have data missing at random, with a model mismatch, and we don't propensity weight the training examples for the regession.

For the impute_missing_w estimator, we're using inverse propensity weighting in fitting the linear regression. We would expect this to potentially decrease the bias of the estimator at the expense of an increase in variance. However, this model has no bias because it perfectly matches the distribution. The bias doesn't seem much bigger than 0 (in terms of SE's), though the SD and RMSE is twice as large than the imputation estimator without IPW.

For this particular distribution, imputing with a linear model works tremendously well. Both the SD and the bias, as well as our two error measures, are the lowest on the board. However, this is largely due to the fact that a linear model is a perfect fit for the data distribution. We'll now repeat this experiment with the modified dataset, in which the conditional mean of y is not a linear function of x. Now Y has mean 1 whether x=1.0 or 2.0, but has mean 0 for x=0.0.

# SeaVan2

In [89]:
sigma = 1
random_seed = 1000
sample_size = 1000
r = default_rng(random_seed)
sampler = get_sampler(dist="SeaVan2", sigma=sigma)
d = sampler(sample_size)
sns.jointplot(data=d, x="x", y="y", kind="reg",x_jitter=.1, scatter_kws={"s": 1})
sns.jointplot(data=d[d["obs"]], x="x", y="y", kind="reg",x_jitter=.1, scatter_kws={"s": 1})
d_big = sampler(n=10000000)
seavan2_full_data_mean = np.mean(d_big['y'])
print(f"Mean of all y's from sample of size {d_big.shape[0]}: {seavan2_full_data_mean} ")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\user\anaconda3\lib\site-packages\seaborn\axisgrid.py:1668: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f = plt.figure(figsize=(height, height))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean of all y's from sample of size 10000000: 0.6660777856749452 


The first plot above is of the full dataset, while the second is from the observed portion of the dataset. An unweighted linear fit is given through each set of points. Note that the fit changes substantially when we leave out the missing data, as opposed to the first data distribution we examined. The hope is that by inverse propensity weighting the observed points in the linear fit, we can get a better approximation to the linear fit of the fully observed data.

In the first scatter plot, we see that the best a linear fit can do with this nonlinear relation. The mismatch of the model to the data gives rise to a bias that we will call model bias. In the second scatter plot, which shows the data we actually observe, we see that the linear fit significantly overestimates the mean of y for x=2. Here the model bias is exacerbated by a sample bias. Here are the results for our various estimators on this dataset.

In [96]:
#expit propensity
num_repeats = 1000  #use 5000 for better SEs
random_seed = 1000
rng = default_rng(random_seed)
sampler = get_sampler(dist="SeaVan2", sigma=sigma, rng = rng)
data_list = []
num_obs_list = []
for i in range(num_repeats):
    ## Randomly choose observations
    d = sampler(n=sample_size)
    est = get_mean_estimators(vals=d['y'],
                              covar=d[["x"]],
                              propensities=d["obs_prob"],
                              obs=d['obs'])
    data_list.append(est)
    num_obs_list.append(d['obs'].sum())
results_seavan2 = pd.DataFrame(data_list)

In [91]:
#log_reg propensity
num_repeats = 1000  #use 5000 for better SEs
random_seed = 1000
rng = default_rng(random_seed)
sampler = get_sampler(dist="SeaVan2", sigma=sigma, rng = rng)
data_list = []
num_obs_list = []
for i in range(num_repeats):
    ## Randomly choose observations
    d = sampler(n=sample_size)
    est = get_mean_estimators_pred(vals=d['y'],
                              covar=d[["x"]],
                              propensities=d["obs_prob_pred"],
                              obs=d['obs'])
    data_list.append(est)
    num_obs_list.append(d['obs'].sum())
results_seavan2_pred = pd.DataFrame(data_list)

In [98]:
d_big = sampler(n=10000000)
seavan2_full_data_mean = np.mean(d_big['y'])
print(f"Mean of all y's from sample of size {d_big.shape[0]}: {seavan2_full_data_mean} ")

Mean of all y's from sample of size 10000000: 0.6667028542738604 


In [99]:
results = pd.concat([results_seavan2, results_seavan2_pred], axis=1)

estimators_to_keep = ["ipw_mean", "ipw_mean_pred", "ipw_b_mean", "ipw_b_mean_pred", "impute_missing", "impute_missing_pred", 
                     "impute_missing_w", "impute_missing_w_pred", "ipw_dr_1", "ipw_dr_1_pred", "ipw_dr_w_1", "ipw_dr_w_1_pred", 
                      "ipw_dr_opt", "ipw_dr_opt_pred", "ipw_dr_w_opt", "ipw_dr_w_opt_pred",
                      "ipw_dr_b", "ipw_dr_b_pred", "ipw_dr_w_b", "ipw_dr_w_b_pred"]
a = results[estimators_to_keep].apply(eval_estimators, axis=0, raw=True, 
                                      result_type="expand", true_val=seavan2_full_data_mean)
results_eval = pd.DataFrame(a.to_dict())
print(results_eval)

              ipw_mean  ipw_mean_pred  ipw_b_mean  ipw_b_mean_pred  \
mean          0.673968       0.586963    0.665035         0.636445   
SD            0.189785       0.125457    0.141190         0.119475   
SE            0.006002       0.003967    0.004465         0.003778   
bias          0.007265      -0.079739   -0.001668        -0.030258   
mean_abs_err  0.152396       0.121636    0.112997         0.097530   
RMS error     0.189924       0.148654    0.141200         0.123247   

              impute_missing  impute_missing_pred  impute_missing_w  \
mean                0.940288             0.940288          0.678548   
SD                  0.079358             0.079358          0.146519   
SE                  0.002510             0.002510          0.004633   
bias                0.273585             0.273585          0.011845   
mean_abs_err        0.273585             0.273585          0.114784   
RMS error           0.284863             0.284863          0.146997   

           

Comparison

The bias is low and SD is high for ipw_mean and ipw_b_mean, the SD of the respective learned estimators have improved but at the cost of increasing the bias. The opposite is the case for the impute_missing estimators. For all the ipw_dr estimators, the learned estimators lower the SD at the price of increasng the bias.


# SeaVanMod2

In [94]:
#expit propensity
num_repeats = 1000  #use 5000 for better SEs
random_seed = 1000
rng = default_rng(random_seed)
sampler = get_sampler(dist="SeaVanMod2", sigma=sigma, rng = rng)
data_list = []
num_obs_list = []
for i in range(num_repeats):
    ## Randomly choose observations
    d = sampler(n=sample_size)
    est = get_mean_estimators(vals=d['y'],
                              covar=d[["x"]],
                              propensities=d["obs_prob"],
                              obs=d['obs'])
    data_list.append(est)
    num_obs_list.append(d['obs'].sum())
results_seavanMod2 = pd.DataFrame(data_list)

In [102]:
#log_reg propensity
num_repeats = 1000  #use 5000 for better SEs
random_seed = 1000
rng = default_rng(random_seed)
sampler = get_sampler(dist="SeaVanMod2", sigma=sigma, rng = rng)
data_list = []
num_obs_list = []
for i in range(num_repeats):
    ## Randomly choose observations
    d = sampler(n=sample_size)
    est = get_mean_estimators_pred(vals=d['y'],
                              covar=d[["x"]],
                              propensities=d["obs_prob_pred"],
                              obs=d['obs'])
    data_list.append(est)
    num_obs_list.append(d['obs'].sum())
results_seavanMod2_pred = pd.DataFrame(data_list)

In [103]:
d_big = sampler(n=10000000)
seavanmod2_full_data_mean = np.mean(d_big['y'])
print(f"Mean of all y's from sample of size {d_big.shape[0]}: {seavanmod2_full_data_mean} ")

Mean of all y's from sample of size 10000000: 1.6676854683793805 


In [104]:
results = pd.concat([results_seavanMod2, results_seavanMod2_pred], axis=1)

estimators_to_keep = ["ipw_mean", "ipw_mean_pred", "ipw_b_mean", "ipw_b_mean_pred", "impute_missing", "impute_missing_pred", 
                     "impute_missing_w", "impute_missing_w_pred"]
a = results[estimators_to_keep].apply(eval_estimators, axis=0, raw=True, 
                                      result_type="expand", true_val=seavanmod2_full_data_mean)
results_eval = pd.DataFrame(a.to_dict())
print(results_eval)

              ipw_mean  ipw_mean_pred  ipw_b_mean  ipw_b_mean_pred  \
mean          1.690247       1.340515    1.641711         1.443464   
SD            0.545853       0.295947    0.340242         0.235740   
SE            0.017261       0.009359    0.010759         0.007455   
bias          0.022561      -0.327170   -0.025975        -0.224222   
mean_abs_err  0.435857       0.366312    0.272981         0.262279   
RMS error     0.546319       0.441163    0.341232         0.325344   

              impute_missing  impute_missing_pred  impute_missing_w  \
mean                1.125976             1.125976          1.654647   
SD                  0.092203             0.092203          0.163546   
SE                  0.002916             0.002916          0.005172   
bias               -0.541709            -0.541709         -0.013038   
mean_abs_err        0.541709             0.541709          0.129804   
RMS error           0.549500             0.549500          0.164065   

           

In [105]:
sns.jointplot(data=d, x="x", y="y", kind="reg",x_jitter=.1, scatter_kws={"s": 1})
sns.jointplot(data=d[d["obs"]], x="x", y="y", kind="reg",x_jitter=.1, scatter_kws={"s": 1})

C:\Users\user\anaconda3\lib\site-packages\seaborn\axisgrid.py:1668: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f = plt.figure(figsize=(height, height))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\user\anaconda3\lib\site-packages\seaborn\axisgrid.py:1668: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f = plt.figure(figsize=(height, height))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# MCAR_normal_sqr

In [106]:
#expit propensity
num_repeats = 1000  #use 5000 for better SEs
random_seed = 1000
sigma = 1
obs_prob = 0.1
rng = default_rng(random_seed)
sampler = get_sampler(dist="MCAR_normal_sqr", sigma=sigma, obs_prob=obs_prob, rng = rng)
data_list = []
num_obs_list = []
for i in range(num_repeats):
    ## Randomly choose observations
    d = sampler(n=sample_size)
    est = get_mean_estimators(vals=d['y'],
                              covar=d[["x"]],
                              propensities=d["obs_prob"],
                              obs=d['obs'])
    data_list.append(est)
    num_obs_list.append(d['obs'].sum())
results_MCAR_normal_sqr = pd.DataFrame(data_list)
d_big = sampler(n=10000000)
MCAR_normal_sqr_full_data_mean = np.mean(d_big['y'])
print(f"Mean of all y's from sample of size {d_big.shape[0]}: {MCAR_normal_sqr_full_data_mean} ")

Mean of all y's from sample of size 10000000: 3.002234668379598 


In [107]:
#log_reg propensity
num_repeats = 1000  #use 5000 for better SEs
random_seed = 1000
sigma = 1
obs_prob = 0.1
rng = default_rng(random_seed)
sampler = get_sampler(dist="MCAR_normal_sqr", sigma=sigma, obs_prob=obs_prob, rng = rng)
data_list = []
num_obs_list = []
for i in range(num_repeats):
    ## Randomly choose observations
    d = sampler(n=sample_size)
    est = get_mean_estimators_pred(vals=d['y'],
                              covar=d[["x"]],
                              propensities=d["obs_prob_pred"],
                              obs=d['obs'])
    data_list.append(est)
    num_obs_list.append(d['obs'].sum())
results_MCAR_normal_sqr_pred = pd.DataFrame(data_list)
d_big = sampler(n=10000000)
MCAR_normal_sqr_full_data_mean_pred = np.mean(d_big['y'])
print(f"Mean of all y's from sample of size {d_big.shape[0]}: {MCAR_normal_sqr_full_data_mean_pred} ")

Mean of all y's from sample of size 10000000: 3.002234668379598 


In [108]:
results = pd.concat([results_MCAR_normal_sqr, results_MCAR_normal_sqr_pred], axis=1)

estimators_to_keep = ["ipw_mean", "ipw_mean_pred", "ipw_b_mean", "ipw_b_mean_pred", "impute_missing", "impute_missing_pred"]
a = results[estimators_to_keep].apply(eval_estimators, axis=0, raw=True, 
                                      result_type="expand", true_val=MCAR_normal_sqr_full_data_mean)
results_eval = pd.DataFrame(a.to_dict())
print(results_eval)

              ipw_mean  ipw_mean_pred  ipw_b_mean  ipw_b_mean_pred  \
mean          2.975643       2.997494    2.982905         2.997883   
SD            0.468419       0.197088    0.364441         0.196488   
SE            0.014813       0.006232    0.011525         0.006213   
bias         -0.026591      -0.004741   -0.019330        -0.004352   
mean_abs_err  0.375287       0.157111    0.289040         0.156575   
RMS error     0.469173       0.197145    0.364953         0.196536   

              impute_missing  impute_missing_pred  
mean                2.993700             2.993700  
SD                  0.196990             0.196990  
SE                  0.006229             0.006229  
bias               -0.008535            -0.008535  
mean_abs_err        0.157176             0.157176  
RMS error           0.197175             0.197175  


Comparison

The learned estimators are much better than the trained ones in terms of both bias and SD.

In [109]:
sns.jointplot(data=d, x="x", y="y", kind="reg",x_jitter=.1, scatter_kws={"s": 1})

C:\Users\user\anaconda3\lib\site-packages\seaborn\axisgrid.py:1668: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f = plt.figure(figsize=(height, height))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …